In [1]:
!git clone https://github.com/manhmitcf/data.git

Cloning into 'data'...
remote: Enumerating objects: 4397, done.
remote: Total 4397 (delta 0), reused 0 (delta 0), pack-reused 4397 (from 1)
Receiving objects: 100% (4397/4397), 177.67 MiB | 45.92 MiB/s, done.
Resolving deltas: 100% (423/423), done.
Updating files: 100% (4394/4394), done.


In [2]:
model_name = 'ResNet50'

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision import transforms
import os
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import pandas as pd
from PIL import Image
from torch.optim.lr_scheduler import StepLR

In [4]:


class FishDatasetWithAugmentation(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, aug_transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.aug_transform = aug_transform
        self.labels = {"Highly Fresh" : 0, "Fresh" : 1, "Not Fresh": 2}  
        # Kiểm tra dữ liệu đầu vào
        if not os.path.exists(img_dir) :
            raise FileNotFoundError(f"Thư mục ảnh '{img_dir}' không tồn tại.")
        if self.data.empty:
            raise ValueError(f"File CSV '{csv_file}' không chứa dữ liệu.")
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        check = False
        img_name = os.path.join(self.img_dir, self.data.iloc[idx, 2])
        if not os.path.exists(img_name):
            check = True
        if check:
            img_name = os.path.join(self.img_dir, self.data.iloc[idx, 2])
            img_name = img_name.replace('_5', '_#')
            if not os.path.exists(img_name):
                raise FileNotFoundError(f"Không tìm thấy ảnh '{img_name}'.")
        try:
            image = Image.open(img_name).convert('RGB')  # Đọc và chuyển đổi ảnh sang RGB
        except FileNotFoundError:
            raise FileNotFoundError(f"Không tìm thấy ảnh '{img_name}'.")


        label = self.data.iloc[idx, 1]
        if label not in self.labels:
            raise ValueError(f"Nhãn '{label}' không hợp lệ. Phải là một trong {list(self.labels.keys())}.")
        label = self.labels[label]
        label = torch.tensor(label, dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        elif self.aug_transform:
            image = self.aug_transform(image)
        else:
            raise ValueError("Cả transform và aug_transform đều là None. Ít nhất một trong hai phải được cung cấp.")

        return image, label
basic_transform = transforms.Compose([
        transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.LANCZOS),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

aug_transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.LANCZOS),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=30),
    transforms.ColorJitter(brightness=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:

from torchvision.models import ResNet50_Weights

class FishClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super(FishClassifier, self).__init__()
        self.resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
    def forward(self, x):
        return self.resnet(x)

In [6]:
import torch
import copy

class EarlyStopping:
    def __init__(self, patience=10, mode="min"):
        """
        patience: số epoch không cải thiện để dừng
        mode: "min" cho val_loss (càng thấp càng tốt), "max" cho val_acc (càng cao càng tốt)
        """
        self.patience = patience
        self.mode = mode
        self.best_loss = float("inf")  # ban đầu giá trị loss rất lớn
        self.best_acc = -float("inf")  # ban đầu giá trị accuracy rất thấp
        self.counter = 0
        self.best_weights = None
        self.early_stop = False

    def check_improvement(self, val_loss, val_acc, model):
        """
        Kiểm tra cải thiện dựa trên cả val_loss và val_acc.
        """
        # Kiểm tra có cải thiện `val_loss` hoặc `val_acc`
        if val_acc > self.best_acc:
            # Nếu `val_loss` giảm hoặc `val_acc` tăng, cập nhật mô hình tốt nhất
            self.best_loss = val_loss
            self.best_acc = val_acc
            self.best_weights = copy.deepcopy(model.state_dict())  # Lưu lại trọng số của mô hình
            self.counter = 0  # Reset counter vì đã có cải thiện
            return True

        # Nếu không có cải thiện
        self.counter += 1
        if self.counter >= self.patience:
            self.early_stop = True  # Nếu không có cải thiện sau `patience` epoch thì dừng
        return False

    def restore_best_model(self, model):
        """Khôi phục lại mô hình tốt nhất"""
        if self.best_weights is not None:
            model.load_state_dict(self.best_weights)
            print("Restored best model weights.")


In [7]:
import time
# Cấu hình
TRAIN_CSV_PATH = "data/train.csv"
VAL_CSV_PATH = "data/val.csv"
IMG_DIR = "data/images/"
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 1e-4
NUM_CLASSES = 3
early_stopper = EarlyStopping(patience=20, mode="min")  # Hoặc "max" nếu bạn muốn theo dõi accuracy
epoch = None 
# Dataset và DataLoader
train_dataset = FishDatasetWithAugmentation(
    csv_file=TRAIN_CSV_PATH,
    img_dir=IMG_DIR,
    transform=None,
    aug_transform=aug_transform,  
)

val_dataset = FishDatasetWithAugmentation(
    csv_file=VAL_CSV_PATH,
    img_dir=IMG_DIR,
    transform=basic_transform,
)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Model + Loss + Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FishClassifier(num_classes=NUM_CLASSES)
model.to(device)
model = torch.nn.DataParallel(model) 

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-2)
# Log để lưu lại loss/acc
loss_train, loss_val = [], []
acc_train, acc_val = [], []
total_start_time = time.time()
epoch_times = []
# Training loop
for epoch in range(EPOCHS):
    epoch_start_time = time.time() 
    # Train
    model.train()
    train_running_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for images, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{EPOCHS} (Train)"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    
    avg_train_loss = train_running_loss / len(train_dataloader)
    train_accuracy = 100 * train_correct / train_total

    # Validation
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    avg_val_loss = val_running_loss / len(val_dataloader)
    val_accuracy = 100 * val_correct / val_total

    # Lưu log
    loss_train.append(avg_train_loss)
    loss_val.append(avg_val_loss)
    acc_train.append(train_accuracy)
    acc_val.append(val_accuracy)

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"    Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"    Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    # Sau khi tính avg_val_loss và val_accuracy
    if early_stopper.check_improvement(avg_val_loss, val_accuracy, model):
        print("Improved! Saving best model.")
    else:
        print(f"No improvement for {early_stopper.counter} epochs.")

    # Nếu không cải thiện trong `patience` epochs, dừng sớm và khôi phục mô hình tốt nhất
    if early_stopper.early_stop:
        print("Early stopping triggered.")
        early_stopper.restore_best_model(model)
        break
    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    epoch_times.append(epoch_duration)  # Lưu thời gian epoch này
# Tính thời gian kết thúc toàn bộ training
total_end_time = time.time()
total_duration = total_end_time - total_start_time
print(f"\n⏳ Total training time: {total_duration/60:.2f} minutes.")
# Trung bình thời gian 1 epoch
avg_epoch_time = sum(epoch_times) / len(epoch_times)
print(f"⏱️ Average time per epoch: {avg_epoch_time:.2f} seconds (~{avg_epoch_time/60:.2f} minutes)")

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 216MB/s]
Epoch 1/100 (Train): 100%|██████████| 44/44 [00:33<00:00,  1.33it/s]


Epoch 1/100
    Train Loss: 0.8609, Train Accuracy: 59.52%
    Val Loss: 0.9902, Val Accuracy: 58.89%
Improved! Saving best model.


Epoch 2/100 (Train): 100%|██████████| 44/44 [00:30<00:00,  1.42it/s]


Epoch 2/100
    Train Loss: 0.6151, Train Accuracy: 72.52%
    Val Loss: 0.7805, Val Accuracy: 68.71%
Improved! Saving best model.


Epoch 3/100 (Train): 100%|██████████| 44/44 [00:31<00:00,  1.41it/s]


...

Epoch 30/100
    Train Loss: 0.0369, Train Accuracy: 99.00%
    Val Loss: 0.7955, Val Accuracy: 78.38%
No improvement for 18 epochs.


Epoch 31/100 (Train): 100%|██████████| 44/44 [00:33<00:00,  1.31it/s]


Epoch 31/100
    Train Loss: 0.0528, Train Accuracy: 98.18%
    Val Loss: 0.9169, Val Accuracy: 77.10%
No improvement for 19 epochs.


Epoch 32/100 (Train): 100%|██████████| 44/44 [00:33<00:00,  1.31it/s]


Epoch 32/100
    Train Loss: 0.0426, Train Accuracy: 98.50%
    Val Loss: 0.8763, Val Accuracy: 76.53%
No improvement for 20 epochs.
Early stopping triggered.
Restored best model weights.

⏳ Total training time: 20.24 minutes.
⏱️ Average time per epoch: 37.95 seconds (~0.63 minutes)


In [8]:
import matplotlib.pyplot as plt
import os

# Đảm bảo thư mục 'result' tồn tại
os.makedirs('result', exist_ok=True)

# Vẽ Loss
plt.figure()
plt.plot(loss_train, label='Train Loss')
plt.plot(loss_val, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()
plt.grid(True)
plt.savefig('result/loss_curve.png')  # Lưu vào file
plt.close()

# Vẽ Accuracy
plt.figure()
plt.plot(acc_train, label='Train Accuracy')
plt.plot(acc_val, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy over Epochs')
plt.legend()
plt.grid(True)
plt.savefig('result/accuracy_curve.png')  # Lưu vào file
plt.close()

In [9]:

torch.save(model.state_dict(), f"result/fish_classifier_{model_name}.pth")
print("Đã lưu mô hình!")

Đã lưu mô hình!


In [10]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import timm
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
import matplotlib.pyplot as plt

In [11]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

# ==== Load dataset test ====
CSV_PATH = "data/test.csv"
IMG_DIR = "data/images/"

try:
    dataset = FishDatasetWithAugmentation(CSV_PATH, IMG_DIR, transform=basic_transform)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
except FileNotFoundError as e:
    raise FileNotFoundError(f"Lỗi khi tải dataset: {e}")

# ==== Dự đoán và tính metrics ====
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds, average="macro")
precision = precision_score(all_labels, all_preds, average="macro")
recall = recall_score(all_labels, all_preds, average="macro")

print(f"✅ Accuracy:  {acc:.4f}")
print(f"✅ F1-score:  {f1:.4f}")
print(f"✅ Precision: {precision:.4f}")
print(f"✅ Recall:    {recall:.4f}")

# ==== Classification Report + Confusion Matrix ====
try:
    class_names = ["Highly Fresh", "Fresh", "Not Fresh"]
    if len(set(all_labels)) > len(class_names):
        raise ValueError("Số lượng lớp thực tế lớn hơn số lớp được định nghĩa.")
    
    print("\n📊 Classification Report:\n")
    print(classification_report(all_labels, all_preds, target_names=class_names))
    
    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    
    # Vẽ và lưu
    fig, ax = plt.subplots(figsize=(8, 6))  # Bạn có thể chỉnh kích thước tùy ý
    disp.plot(cmap="Blues", values_format="d", ax=ax)
    plt.title("Confusion Matrix")
    plt.savefig('result/confusion_matrix.png', dpi=300)  # Lưu vào file PNG
    plt.close()  # Đóng plot để không bị chồng hình khi vẽ tiếp

    print("✅ Đã lưu confusion matrix vào thư mục 'result/'.")

except ValueError as e:
    print(f"⚠️ Lỗi trong việc tạo báo cáo lớp: {e}")

✅ Accuracy:  0.8098
✅ F1-score:  0.8013
✅ Precision: 0.8020
✅ Recall:    0.8008

📊 Classification Report:

              precision    recall  f1-score   support

Highly Fresh       0.86      0.88      0.87       366
       Fresh       0.74      0.72      0.73       271
   Not Fresh       0.80      0.80      0.80       241

    accuracy                           0.81       878
   macro avg       0.80      0.80      0.80       878
weighted avg       0.81      0.81      0.81       878

✅ Đã lưu confusion matrix vào thư mục 'result/'.


In [12]:
optimize = "AdamW"

In [13]:
results = {
    "model_name": model_name,
    "optimizer": optimize,
    "lr": LEARNING_RATE,
    "batch_size": BATCH_SIZE,
    "epochs": EPOCHS,
    "epochs_current": epoch,
    "val_best_acc": early_stopper.best_acc * 0.01,
    "val_best_loss": early_stopper.best_loss,
    "accuracy": acc,
    "precision": precision,
    "recall": recall,
    "f1_score": f1,
    "time(s)": total_duration,
    "time_per_epoch(s)": avg_epoch_time
}

In [14]:
results_df = pd.DataFrame([results])
results_df.to_csv(f"result/evaluation_results_{model_name}.csv", index=False)

In [15]:
import json
import os
from datetime import datetime

def save_model_config(save_dir,config):

    # Xóa các trường None để file json gọn
    config = {k: v for k, v in config.items() if v is not None}

    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{model_name}_config.json")
    
    with open(save_path, 'w') as f:
        json.dump(config, f, indent=4)
    
    print(f"✅ Đã lưu file config: {save_path}")


In [16]:
    config = {
        "model_name": model_name,
        "input_size": 224,
        "num_classes": 3,
        "batch_size": BATCH_SIZE,
        "learning_rate": LEARNING_RATE,
        "optimizer": optimize,
        "loss_function": "CrossEntropyLoss",
        "weight_decay": 0.02,
        "epoch_trained": epoch,
        "best_val_acc": early_stopper.best_acc * 0.01,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "loss_train": loss_train,
        "loss_val": loss_val,
        "acc_train": acc_train,
        "acc_val": acc_val
    }

In [17]:
save_model_config(save_dir="result",config = config)

✅ Đã lưu file config: result/ResNet50_config.json


In [18]:
from IPython.display import FileLink
import shutil
import os

# Giả sử model_name đã được định nghĩa
zip_filename = f"result_{model_name}.zip"

# Nén folder
shutil.make_archive(base_name=zip_filename.replace('.zip', ''), format='zip', root_dir='result')

# Tạo link tải
print("✅ File đã sẵn sàng để tải:")
display(FileLink(zip_filename))


✅ File đã sẵn sàng để tải:


/kaggle/working/result_ResNet50.zip